In [1]:
import numpy as np
import xarray as xr
import copy 

In [ ]:
#Use CDO for regridding data to MPI-GE grid
#cdo -remapbil,MPI_GE_grid.txt in_file_original.nc out_file_GEgrid.nc

In [2]:
def window_stack(a, stepsize, window):
    return np.hstack( a[i:1+i-window or None:stepsize] for i in range(0,window) )


In [3]:
def r_anomaly(in_data,r_window):
    r_window_data = window_stack(in_data,1,window) #get data for running windows
    r_mean = r_window_data.mean(axis=1) #average over the each window
    
    #subtract mean to original data
    start = int((window-1)/2)
    in_data[:start]= in_data[:start]-r_mean[0] #first missing values
    in_data[-start:] = in_data[-start:]-r_mean[-1] #last missing values
    in_data[start:-start] = in_data[start:-start,:]-np.repeat(r_mean[:,np.newaxis],in_data.shape[1],axis=1) #middle values
    return(in_data)

In [4]:
#Mask
path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/'
file = 'nGE_Tmax_ssp245_dm_land_europe.nc'
mask = xr.open_dataset(path+file)
mask = mask.tasmax[0,0,:,:]

In [5]:
stepsize = 1
window = 15
start = int((window-1)/2)

In [6]:
model = 'MIROC6'
ens_number = '50'

In [7]:
run = ['historical','ssp245','ssp585']

In [8]:
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model+'/tasmax/'
    file = model+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_europe.nc'
    
    with xr.open_dataset(path+file) as data:
        data = data.tasmax
        data2 = np.where(mask==0,0,data) #mask ocean data
        data[:,:,:,:] = data2
        
    #flatten lon/lat
    data_flat = data.values.reshape(data.shape[0],data.shape[1],-1) 
    
    #calculate running window ensemble mean and anomalies
    r_anomaly_output = copy.deepcopy(data_flat)
    for x in range(data_flat.shape[2]):
        in_data = data_flat[:,:,x]
        r_anomaly_output[:,:,x] = r_anomaly(in_data,window)
        #print(x)
        
    r_anomaly_output = r_anomaly_output.reshape(data.shape[0],data.shape[1],data.shape[2],data.shape[3])
    
    # ------------------ save data -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model+'/tasmax/'
    file = model+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_land_europe_anomaly_detrend.nc'
    data[:,:,:,:] = r_anomaly_output[:,:,:,:]
    data.to_netcdf(path+file)
    
    print(n_run + ' finished')

/tmp/ipykernel_463326/2570541340.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.hstack( a[i:1+i-window or None:stepsize] for i in range(0,window) )


historical finished
ssp245 finished


/tmp/ipykernel_463326/2570541340.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.hstack( a[i:1+i-window or None:stepsize] for i in range(0,window) )


ssp585 finished
